In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.25"
# os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
# from desc import set_device
# set_device("gpu")

In [2]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [3]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic
from desc.compute.data_index import register_compute_fun
from desc.optimize.utils import solve_triangular_regularized

print_backend_info()

DESC version=0.15.0+493.g32d73fe16.
Using JAX backend: jax version=0.6.2, jaxlib version=0.6.2, dtype=float64.
Using device: CPU, with 6.79 GB available memory.


In [4]:
from desc.particles import *
from diffrax import *

In [5]:
from scipy.io import netcdf_file

def vmec_to_cylindrical(wout_filename, s_vmec, theta_vmec, phi_vmec):
    r"""
    Convert from VMEC coordinates to cylindrical coordinates.

    Args:
        wout_filename : str
            The name of the VMEC wout file.
        s_vmec : A scalar or a numpy array of shape (npoints,) containing the
            normalized toroidal flux.
        theta_vmec : A scalar or a numpy array of shape (npoints,) containing
            the VMEC poloidal angle.
        phi_vmec : A scalar or a numpy array of shape (npoints,) containing
            the VMEC cylindrical angle.

    Returns:
        R : A scalar or a numpy array of shape (npoints,) containing the
            radial coordinate.
        phi_cyl : A scalar or a numpy array of shape (npoints,) containing
            the azimuthal angle.
        Z : A scalar or a numpy array of shape (npoints,) containing the
            vertical coordinate.
    """
    # Handle scalar inputs - return scalars if any input is a scalar
    input_scalar = (
        np.isscalar(s_vmec) or np.isscalar(theta_vmec) or np.isscalar(phi_vmec)
    )

    # Convert to arrays for processing
    s_vmec = np.asarray(s_vmec)
    theta_vmec = np.asarray(theta_vmec)
    phi_vmec = np.asarray(phi_vmec)

    # Validate that arrays are not empty
    if s_vmec.size == 0:
        raise ValueError("Input arrays cannot be empty")

    # Load VMEC data
    with netcdf_file(wout_filename, "r") as f:
        rmnc = f.variables["rmnc"][:]  # R harmonics (cos)
        zmns = f.variables["zmns"][:]  # Z harmonics (sin)
        xm = f.variables["xm"][:]  # poloidal mode numbers
        xn = f.variables["xn"][:]  # toroidal mode numbers
        ns = int(f.variables["ns"][()])  # number of radial surfaces (scalar)
        s_full = np.linspace(0, 1, ns)  # full radial grid

    # Initialize R and Z arrays
    R = np.zeros_like(s_vmec)
    Z = np.zeros_like(s_vmec)

    # For each point, compute R and Z using VMEC Fourier harmonics
    for i in range(s_vmec.size):
        s_i = s_vmec[i]
        theta_i = theta_vmec[i]
        phi_i = phi_vmec[i]

        # Interpolate harmonics to the desired s value
        rmnc_s = np.zeros_like(rmnc[0, :])
        zmns_s = np.zeros_like(zmns[0, :])

        for j in range(rmnc.shape[1]):  # Loop over mode numbers
            # Interpolate rmnc and zmns to s_i
            rmnc_s[j] = np.interp(s_i, s_full, rmnc[:, j])
            zmns_s[j] = np.interp(s_i, s_full, zmns[:, j])

        # Compute R and Z using Fourier series
        R[i] = 0.0
        Z[i] = 0.0

        for j in range(len(xm)):
            angle = xm[j] * theta_i - xn[j] * phi_i
            R[i] += rmnc_s[j] * np.cos(angle)
            Z[i] += zmns_s[j] * np.sin(angle)

    # phi_cyl is the same as phi_vmec
    phi_cyl = phi_vmec

    # Return scalars for scalar inputs, arrays for array inputs
    if input_scalar:
        return (
            R[0] if hasattr(R, "__len__") else R,
            phi_cyl[0] if hasattr(phi_cyl, "__len__") else phi_cyl,
            Z[0] if hasattr(Z, "__len__") else Z,
        )
    else:
        return R, phi_cyl, Z

In [10]:
from desc.vmec import VMECIO
wout_filename = "booz_xform/wout_LandremanPaul2021_QH_reactorScale_lowres_reference.nc"
eq = VMECIO.load(wout_filename, L=8, M=8, N=8, profile="current")

In [23]:
phi_vmec = np.loadtxt("initial_condition/phi_vmec.txt")
s_booz = np.loadtxt("initial_condition/s_booz.txt")
theta_vmec = np.loadtxt("initial_condition/theta_vmec.txt")
vartheta_booz = np.loadtxt("initial_condition/vartheta_booz.txt")
zeta_booz = np.loadtxt("initial_condition/zeta_booz.txt")

In [24]:
phi_vmec = np.atleast_1d(phi_vmec)
s_booz = np.atleast_1d(s_booz)
theta_vmec = np.atleast_1d(theta_vmec)
vartheta_booz = np.atleast_1d(vartheta_booz)
zeta_booz = np.atleast_1d(zeta_booz)

In [25]:
coords_booz = jnp.array([s_booz, vartheta_booz, zeta_booz]).T
coords_vmec = jnp.array([s_booz, theta_vmec, phi_vmec]).T

In [26]:
R_init, phi_init, Z_init = vmec_to_cylindrical(wout_filename, s_booz, theta_vmec, phi_vmec)

/home/yigit/miniconda3/envs/desc-env/lib/python3.12/site-packages/scipy/io/_netcdf.py:323: RuntimeWarning:

Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)



In [27]:
traj_vmec = np.loadtxt("run_firm3d/precise_QH/trajectory_data_vmec_tol_1e-10_resolution_96_tmax_0.001_trapped.txt")

In [29]:
R, phi, Z = vmec_to_cylindrical(wout_filename, traj_vmec[:, 1], traj_vmec[:, 2], traj_vmec[:, 3])

/home/yigit/miniconda3/envs/desc-env/lib/python3.12/site-packages/scipy/io/_netcdf.py:323: RuntimeWarning:

Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)



In [30]:
X = R * np.cos(phi)
Y = R * np.sin(phi)

In [31]:
fig = plot_3d(eq, "|B|", alpha=0.4)
fig.add_trace(
    go.Scatter3d(
        x=X,
        y=Y,
        z=Z,
        mode="lines",
        line=dict(
            color="red",
            width=5,
            dash="solid",
        ),
        marker=dict(size=0),
        showlegend=False,
    )
)

/home/yigit/Codes/DESC/desc/utils.py:572: UserWarning:

Unequal number of field periods for grid 1 and basis 4.

/home/yigit/Codes/DESC/desc/utils.py:572: UserWarning:

Unequal number of field periods for grid 1 and basis 4.

